In [1]:
import time
import numpy as np

#유틸

In [2]:
def _t(x):
    return np.transpose(x)
def _m(A,B):
    return np.matmul(A,B)

In [9]:
#전과 다르게 class로 하는 이유는 dp를 위한 메모리확보를 위해

class Sigmoid:
    def __init__(self):
        self.last_o=1#마지막 출력을 저장하게 해줌
    def __call__(self,x):
        self.last_o = 1/(1.0+np.exp(-x))#미리저장
        return self.last_o

    def grad(self):#미분하면 sig/(1-sig)
        return self.last_o*(1-self.last_o)

In [4]:
class MeanSquareError:
    def __init__(self):
        #grad
        self.dh = 1#마지막 출력을 저장하게 해줌

        #for bp
        self.last_diff = 1
    def __call__(self,h,y):#mse는 1/2*mean(h-y)^2
        self.last_diff = h-y
        return 1/2*np.mean(np.square(h-y))
    def grad(self):#h-y
        return self.last_diff

In [44]:
class Neuron:
    #말만 뉴런이지 layer한 층(벡터화 시켰으니까)
    def __init__(self,W,b,a_obj):
        self.W = W
        self.b = b
        self.a = a_obj()

        #grad
        self.dW = np.zeros_like(self.W)
        self.db = np.zeros_like(self.b)
        #dh = Mx1
        self.dh = np.zeros_like(_t(self.W))
        #print("_t(self.W):{}".format(self.dh.shape))
        self.last_x = np.zeros((self.W.shape[0]))#마지막x를 저장 W를 미분하면 h layer하나 나옴. 즉 for grad_W
        #print("last_x.shape:{}".format(self.last_x.shape))


    def __call__(self,x):
        self.last_x = x
        self.last_h = _m(_t(self.W),x)+self.b
        return self.a(self.last_h)


    #y = w^Tx+b
    #각 layer의 입력과 출력사이의 gradient
    def grad(self):# dy/dh = W, h는 한 layer층의 벡터를 말함
        #print("W*a shape : {}".format((self.W*self.a.grad()).shape))
        return self.W*self.a.grad()

    def grad_W(self, dh):#dh는 누적되서 온 gradient
        #print("W dh shape : {}".format(dh.shape))
        grad = np.ones_like(self.W)
        grad_a = self.a.grad()
        for j in range(grad.shape[1]):#출력 쪽 dimension , y = w^Tx+b에서 w로 편미분이라 x만 남음 dy/dw = x
            grad[:,j] = dh[j]*grad_a[j]*self.last_x#전의 layer부터 내려오는 값들을 저장, 그 뒤는 acti미분 * h (n-1)
        return grad

    def grad_b(self, dh):# dy/dh = 1
        #print("b dh shape : {}".format(dh.shape))
        return dh * self.a.grad()*1

In [17]:
class DNN:
    def __init__(self,hidden_depth, num_neuron, num_input, num_output, activation=Sigmoid):
        def init_var(i, o):#input과 output의 뉴런의 개수
            return np.random.normal(0.0, 0.01, (i,o)), np.zeros((o,))#뒤는 bias

        self.sequence = list()
        #1st
        W, b = init_var(num_input,num_neuron)
        self.sequence.append(Neuron(W,b,activation))

        #other hidden
        for _ in range(hidden_depth):
            W, b = init_var(num_neuron, num_neuron)
            self.sequence.append(Neuron(W,b,activation))

        #output
        W, b = init_var(num_neuron,num_output)
        self.sequence.append(Neuron(W,b,activation))

    def __call__(self,x):
        for layer in self.sequence:#시퀀스의 레이어를 호출해서 반복
            x = layer(x)
        return x

    def calc_gradient(self,loss_obj):
        #gradient 구현
        #핵심!!
        #전과 다르게 forward inference한번으로 저장된 값을 통해한다.
        loss_obj.dh = loss_obj.grad()
        self.sequence.append(loss_obj)
        #이렇게하면 loss가 있어 call할 때 출력을 못 얻고 loss만 얻는다.
        #back propagation
        for i in range(len(self.sequence) - 1,0,-1):#이러면 역순으로 나온다.for문이랑 똑같음
            #하나씩 진행할거다.
            l1 = self.sequence[i]
            l0 = self.sequence[i-1]
            #l1는 전꺼
            l0.dh = _m(l0.grad(), l1.dh)#loss를 현재 layer로 미분한 grad계산된다.
            l0.dW = l0.grad_W(l1.dh)
            l0.db = l0.grad_b(l1.dh)

        self.sequence.remove(loss_obj)

#경사 하강

In [18]:
def gradient_descent(network,x,y,loss_obj, alpha = 0.01):
  loss = loss_obj(network(x),y)
  network.calc_gradient(loss_obj)
  for layer in network.sequence:
    layer.W += -alpha * layer.dW
    layer.b += -alpha * layer.db
  return loss

#실행

##진짜 돌리자마자 되는정도의 차이!!!전엔 86초였다.

In [45]:
x = np.random.normal(0.0, 1.0, (10,))
y = np.random.normal(0.0, 1.0, (2,))

t = time.time()
dnn = DNN(hidden_depth=5, num_neuron=32, num_input=10, num_output=2, activation=Sigmoid)
loss_obj = MeanSquareError()
for epoch in range(100):
    loss = gradient_descent(dnn, x, y, loss_obj, alpha=0.01)
    print('Epoch {}: Test loss {}'.format(epoch, loss))
print('{} seconds elapsed.'.format(time.time() - t))

last_x.shape:(10,)
last_x.shape:(32,)
last_x.shape:(32,)
last_x.shape:(32,)
last_x.shape:(32,)
last_x.shape:(32,)
last_x.shape:(32,)
Epoch 0: Test loss 0.7266304133699091
Epoch 1: Test loss 0.7184334380747983
Epoch 2: Test loss 0.7103327575727756
Epoch 3: Test loss 0.702332953114027
Epoch 4: Test loss 0.694438229400483
Epoch 5: Test loss 0.6866524108545641
Epoch 6: Test loss 0.6789789408978141
Epoch 7: Test loss 0.6714208840618043
Epoch 8: Test loss 0.663980930723616
Epoch 9: Test loss 0.6566614042348395
Epoch 10: Test loss 0.6494642701963045
Epoch 11: Test loss 0.6423911476204045
Epoch 12: Test loss 0.6354433217185315
Epoch 13: Test loss 0.6286217580521631
Epoch 14: Test loss 0.6219271177919747
Epoch 15: Test loss 0.6153597738392053
Epoch 16: Test loss 0.6089198275767154
Epoch 17: Test loss 0.6026071260329707
Epoch 18: Test loss 0.596421279259898
Epoch 19: Test loss 0.5903616777445224
Epoch 20: Test loss 0.5844275096939394
Epoch 21: Test loss 0.5786177780529794
Epoch 22: Test loss 0.5